In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fiddlehead-competition/sample.csv
/kaggle/input/fiddlehead-competition/train.csv
/kaggle/input/fiddlehead-competition/test.csv


In [2]:
#Load basic libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
import sklearn
import xgboost
import imblearn
import numpy as np
import pandas as pd
import scipy
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
#analyse submission sample dataset

sample=pd.read_csv('/kaggle/input/fiddlehead-competition/sample.csv')
sample.head()

,category,id
0,1,0
1,0,1
2,0,2
3,1,3
4,0,4


In [5]:
#load the training and testing datasets

train_data=pd.read_csv('/kaggle/input/fiddlehead-competition/train.csv')
test_data=pd.read_csv('/kaggle/input/fiddlehead-competition/test.csv')

In [6]:
train_data.head()

,category,head shape,head surface,head shade,marks,fragrance,blade_exist,blade_gap,blade_size,blade texture,...,shroud color,annulus number,annulus type,powder color,tot_popul,environment,no_days,head_circumference,stem width,root width
0,1,5,2,4,1,5,1,0,1,4,...,2,1,4,1,3,0,29.0,4.516383,4.591127,5.110601
1,0,2,0,4,1,0,1,1,1,5,...,2,1,2,1,4,0,22.0,5.922174,4.979566,5.341753
2,0,5,2,9,0,5,1,0,0,7,...,2,1,2,7,5,2,23.0,7.089155,3.623844,7.488063
3,1,5,3,0,0,5,1,0,1,10,...,2,2,4,7,4,0,47.0,11.718708,7.452025,4.325183
4,1,5,0,3,0,6,1,1,0,0,...,2,1,2,2,5,2,9.0,8.795063,8.792288,4.309965


In [7]:
train_data.isna().sum()

category              0
head shape            0
head surface          0
head shade            0
marks                 0
fragrance             0
blade_exist           0
blade_gap             0
blade_size            0
blade texture         0
stipe shape           0
base_stipe            0
stem surface above    0
stem surface below    0
stem color above      0
stem color below      0
shroud type           0
shroud color          0
annulus number        0
annulus type          0
powder color          0
tot_popul             0
environment           0
no_days               0
head_circumference    0
stem width            0
root width            0
dtype: int64

In [8]:
#data distribution of target

train_data['category'].value_counts()

0    24529
1    23143
Name: category, dtype: int64

In [9]:
for c in train_data.columns:
  print(train_data[c].value_counts())
  print('--------------')

0    24529
1    23143
Name: category, dtype: int64
--------------
5    21295
2    18684
3     4742
0     2755
4      168
1       28
Name: head shape, dtype: int64
--------------
3    19026
2    14880
0    13755
1       11
Name: head surface, dtype: int64
--------------
4    13419
3    10619
2     8748
9     6288
8     6199
0     1163
5      816
1      248
6      118
7       54
Name: head shade, dtype: int64
--------------
0    27809
1    19863
Name: marks, dtype: int64
--------------
5    20092
2    13095
8     3542
7     3315
0     2483
3     2233
6     1605
1     1128
4      179
Name: fragrance, dtype: int64
--------------
1    46475
0     1197
Name: blade_exist, dtype: int64
--------------
0    40222
1     7450
Name: blade_gap, dtype: int64
--------------
0    33052
1    14620
Name: blade_size, dtype: int64
--------------
0     10064
7      8903
10     7022
5      6393
2      4279
3      4203
9      2732
4      2417
1       568
11      540
6       377
8       174
Name: blade texture

In [10]:
from sklearn.model_selection import train_test_split
train_set,test_set=train_test_split(train_data,test_size=0.2,random_state=42)

In [11]:
X_train=train_set.drop('category',axis=1)
y_train=train_set['category'].copy()

X_test=test_set.drop('category',axis=1)
y_test=test_set['category'].copy()

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(X_train)
scaler.fit_transform(X_test)

array([[0.6       , 0.66666667, 0.        , ..., 0.10309744, 0.59721278,
        0.09753442],
       [0.4       , 0.        , 0.44444444, ..., 0.05257417, 0.45628265,
        0.87692087],
       [1.        , 0.66666667, 1.        , ..., 0.02965401, 0.79900586,
        0.5417838 ],
       ...,
       [1.        , 1.        , 0.88888889, ..., 0.17040302, 0.63654769,
        0.07145691],
       [0.4       , 0.        , 0.22222222, ..., 0.49929366, 0.27889783,
        0.44322464],
       [1.        , 0.66666667, 1.        , ..., 0.82049331, 0.9559465 ,
        0.54838694]])

In [13]:
from sklearn.feature_selection import RFE
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train, y_train)
selector.support_

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


array([False, False, False,  True, False,  True,  True,  True, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False])

In [14]:
selector.ranking_


array([18, 14, 21,  1, 12,  1,  1,  1,  9,  5,  6,  4,  3, 13, 11, 22,  2,
        1,  7, 15,  8, 10, 19, 17, 20, 16])

In [15]:
# from sklearn.linear_model import LogisticRegression
# log_reg = LogisticRegression(random_state=0).fit(X_train, y_train)
# print(log_reg.predict(X_test))
# log_reg.score(X_test, y_test)

In [16]:
# from sklearn.naive_bayes import BernoulliNB
# bern_nb = BernoulliNB().fit(X_train, y_train)
# print(bern_nb.predict(X_test))
# bern_nb.score(X_test, y_test)

In [17]:
# from sklearn.ensemble import AdaBoostClassifier
# ada_boost = AdaBoostClassifier().fit(X_train, y_train)
# print(ada_boost.predict(X_test))
# print(ada_boost.predict_proba(X_test))
# ada_boost.score(X_test, y_test)

In [18]:
# from sklearn.ensemble import BaggingClassifier
# bagging = AdaBoostClassifier(n_estimators=100, random_state=0).fit(X_train, y_train)
# print(bagging.predict(X_test))
# print(bagging.predict_proba(X_test))
# bagging.score(X_test, y_test)

In [19]:
# from sklearn.ensemble import ExtraTreesClassifier
# extra_tree = ExtraTreesClassifier()
# extra_tree.fit(X_train, y_train)
# print(extra_tree.predict(X_test))
# extra_tree.score(X_test,y_test)

In [20]:
# from sklearn.ensemble import GradientBoostingClassifier
# grad_boost = GradientBoostingClassifier()
# grad_boost.fit(X_train, y_train)
# print(grad_boost.predict(X_test))
# grad_boost.score(X_test,y_test)

In [21]:
# rand_forest = RandomForestClassifier(random_state=42).fit(X_train, y_train)
# print(rand_forest.predict(X_test))
# print(rand_forest.predict_proba(X_test))
# rand_forest.score(X_test, y_test)

In [22]:
# from sklearn.ensemble import StackingClassifier
# estimators = [('rf', RandomForestClassifier()),('gbr', GradientBoostingClassifier())]
# stack = StackingClassifier(estimators=estimators, final_estimator=BaggingClassifier())
# stack.fit(X_train, y_train).score(X_test, y_test)

In [23]:
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# clf1 = LogisticRegression(random_state=42)
# clf2 = RandomForestClassifier(random_state=42)
# clf3 = GradientBoostingClassifier(random_state=42)
# voting=VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gbc', clf3)], voting='hard')
# voting = voting.fit(X_train, y_train)
# print(voting.predict(X_test))
# voting.score(X_test, y_test)

In [24]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, ExtraTreesClassifier
#clf1 = LogisticRegression(random_state=42)
clf2 = RandomForestClassifier(random_state=42)
clf3 = GradientBoostingClassifier(random_state=42)
clf4 = ExtraTreesClassifier(random_state=42)
voting=VotingClassifier(estimators=[('rf', clf2), ('gbc', clf3),('extrac', clf4)], voting='hard')
voting = voting.fit(X_train, y_train)
print(voting.predict(X_test))
voting.score(X_test, y_test)

[0 0 0 ... 0 1 1]


0.5824855794441531

In [25]:
# from sklearn.neural_network import MLPClassifier
# MLP = MLPClassifier().fit(X_train, y_train)
# print(MLP.predict(X_test))
# MLP.score(X_test, y_test)

In [26]:
# rand_forest_revised = RandomForestClassifier()
# n_estimators = [100,150,200]
# max_features = ['sqrt', 'log2']
# max_depth=[2,5,10]
# min_samples_split=[2,4,8]
# # define grid search
# grid = dict(n_estimators=n_estimators,max_features=max_features,max_depth=max_depth,min_samples_split=min_samples_split)
# cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=2, random_state=1)
# grid_search = GridSearchCV(estimator=rand_forest_revised, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
# grid_result = grid_search.fit(X_train, y_train)
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#  print("%f (%f) with: %r" % (mean, stdev, param))

In [27]:
y_pred = voting.predict(test_data)
y_pred

array([0, 1, 1, ..., 0, 0, 0])

In [28]:
submission=pd.DataFrame(columns=['category','id'])
submission['id']=[i for i in range(len(y_pred))]

submission['category']=y_pred

submission.to_csv('submission.csv',index=False)

submission

,category,id
0,0,0
1,1,1
2,1,2
3,0,3
4,1,4
...,...,...
20427,0,20427
20428,0,20428
20429,0,20429
20430,0,20430


In [29]:
# KNN = KNeighborsClassifier(n_neighbors=2)
# KNN.fit(X_train, y_train)
# print(KNN.predict(X_test))
# print(KNN.predict_proba(X_test))
# KNN.score(X_test,y_test)

In [30]:
# from sklearn.svm import SVC
# SVC = SVC()
# SVC.fit(X_train, y_train)
# print(SVC.predict(X_test))
# SVC.score(X_test,y_test)